In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np
from matplotlib import pyplot as plt
import os
import json
from datetime import datetime, date
from dateutil.parser import parse
from dotenv import load_dotenv

from NLP_tools import Cleaning_text, top_keywords, top_entities, get_topic_name, best_document, clean_all, topic_documents
from core.functions import *

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from tqdm import tqdm

from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer

from opensearch_data_model import Topic, TopicKeyword, News, os_client, TOPIC_INDEX_NAME, NEWS_INDEX_NAME
from opensearch_io import init_opensearch, get_news
from opensearchpy import helpers

In [2]:
init_opensearch()

Indice Topic creado
El índice News ya existe. Saltando inicialización de base de datos.


In [3]:
load_dotenv()
PATH_REMOTO='/content/ITBA-NLP/data/'
PATH=os.environ.get('PATH_LOCAL', PATH_REMOTO)
PATH

if PATH == os.environ.get('PATH_LOCAL'):
    client = OpenAI(api_key= os.environ.get('OPENAI_API_KEY'))
else:
    from google.colab import userdata
    client = OpenAI(api_key= userdata.get('OPENAI_API_KEY'))

# Para testear app streamlit

In [113]:
# Modelo 1
df_params = {'0_1000':'0_1000_data.parquet',
             '1000_2000':'1000_2000_data.parquet',
             '2000_3000':'2000_3000_data.parquet',
             'df_joined':'df_joined_2024-04-01 00_00_00.parquet'
            }

chunk = '0_1000' 

df_parquet_1 = pd.read_parquet(PATH+df_params[chunk])
#data_1 = list(df_parquet_1['in__text'])

#chunk = "app_1"
# Cargar modelo entrenado o saltar celda y entrenar
#topic_model_1 = BERTopic.load(PATH+f"modelos/bertopic_model_{chunk}")

# Cargar los embeddings
#docs_embedding_1 = np.load(PATH+f"modelos/docs_embeddings_{chunk}.npy")



In [114]:
df_parquet_1.columns

Index(['Asset Name', 'Author Id', 'Author Name', 'Keyword Id', 'Keyword Name',
       'Entity Id', 'Entity Name', 'Media Group Id', 'Media Group Name',
       'Impact', 'in__title', 'in__text', 'out__entities',
       'out__potential_entities', 'predicted_at_entities',
       'out__keywords_sorted', 'predicted_at_keywords', 'start_time_utc',
       'start_time_local', 'truncated_text', 'title_and_text'],
      dtype='object')

In [115]:
df_select = df_parquet_1[["in__title","in__text","start_time_local","Author Name","out__entities","out__keywords_sorted"]]
df_select.columns

Index(['in__title', 'in__text', 'start_time_local', 'Author Name',
       'out__entities', 'out__keywords_sorted'],
      dtype='object')

In [116]:
df_select.reset_index(inplace=True)
df_select.head(1)

,Asset Id,in__title,in__text,start_time_local,Author Name,out__entities,out__keywords_sorted
0,105628101,Elecciones en Venezuela: María Corina Machado pidió más apoyo de Noruega para garantizar unas presidenciales libres,"Fotografía de archivo de la líder antichavista María Corina Machado (EFE/ Rayner Peña R.)\n\nLa líder opositora María Corina Machado pidió a Noruega -país mediador en las negociaciones entre el régimen de Venezuela y la oposición mayoritaria- extremar su apoyo para garantizar que en el país caribeño se celebren, el próximo 28 de julio, unas presidenciales “libres y justas”, según una comunicación difundida este lunes, que insta a que presione al dictador Nicolás Maduro para que cumpla con el Acuerdo de Barbados y admita la candidatura presidencial de su sucesora, Corina Yoris.\n\nLa carta, dirigida al primer ministro noruego, Jonas Gahr, tiene el objetivo de exhortar a su Gobierno, en “su calidad de facilitador del proceso de diálogo y negociación entre actores políticos de Venezuela, a extremar los recursos diplomáticos a su disposición” para que el acuerdo de Barbados -firmado por ambas partes en octubre, que establece garantías electorales- sea “cumplido integralmente”.\n\nMachado, ganadora de las primarias opositoras del pasado octubre, aseguró en el escrito que el acuerdo “ha sido integralmente violado por parte del régimen que preside Nicolás Maduro”.\n\nEn concreto, Machado ha citado una serie de violaciones “flagrantes” del Acuerdo de Barbados, como su inhabilitación por parte de la Justicia venezolana, la detención de al menos ocho personas vinculadas a su equipo electoral, así como el corte de electricidad en la Embajada argentina.\n\n“Maduro ha calificado a mi partido, Vente Venezuela, de ‘organización terrorista’. Para acusarnos han forjado pruebas y forzado a algunos detenidos a acusar a sus propios compañeros de falsas conspiraciones armadas”, señala, en alusión a las declaraciones bajo custodia de su jefe de campaña en Barinas,\n\nAsimismo, ha asegurado que tanto su equipo como ella misma “corren el riesgo de próximas desapariciones forzadas”.\n\nFOTO ARCHIVO. El canciller de México, Marcelo Ebrard, saluda al jefe de la delegación opositora de Venezuela, Gerardo Blyde Pérez, acompañado por el representante del gobierno de Noruega, Dag Nylander, y el presidente de la Asamblea Nacional de Venezuela, Jorge Rodríguez, durante las conversaciones en Ciudad de México (REUTERS/Henry Romero)\n\n“Yo misma podría ser objeto de una detención injustificada”, resalta, agregando que “el pueblo venezolano está dispuesto a luchar con el objetivo” de que se materialice la candidatura de Corina Yoris o la suya misma.\n\n“Con el sólido apoyo internacional que hemos tenido, lograr estas elecciones está al alcance de nuestras manos. Aún hay tiempo para sobreponernos a los impedimentos y obstáculos que Nicolás Maduro ha impuesto con el establecimiento de plazos a su antojo”, ha agregado.\n\nLa semana pasada Machado, sobre quien pesa una inhabilitación de 15 años, anunció su retirada y presentó a Corina Yoris, que fue miembro de la Comisión Nacional de Primaria (CNP), como su sustituta, después de que no pudiera acceder al sistema para inscribir su candidatura ante el Consejo Nacional Electoral (CNE). No obstante, el plazo cerró sin que Yoris pudiera registrarse.\n\nEl CNE confirmó el listado definitivo de los candidatos para las elecciones presidenciales y en él figuran los nombres de Edmundo González Urrutia como aspirante de Mesa de la Unidad Democrática, absorbida por la Plataforma Unitaria Democrática (PUD), y de Manuel Rosales , del partido Un Nuevo Tiempo (UNT), también integrado en la PUD.\n\n(Con información de EFE y EP)",2024-04-01 21:00:00,Infobae,"[Nicolás Maduro, Marcelo Ebrard, Jorge Rodríguez, Consejo Nacional Electoral, CNE]","[elecciones presidenciales, candidatura presidencial, apoyo internacional, inhabilitación, régimen, detención injustificada, sustituta, falsas conspiraciones armadas, sucesora, i

In [117]:
new_column_names = {
    'Asset Id': 'asset_id',
    'in__title': 'title',
    'in__text': 'text',
    'Author Name':'media',
    'out__entities': 'entities',
    'out__keywords_sorted': 'keywords'
}

In [118]:
df_select.rename(columns=new_column_names, inplace=True)


In [119]:
df_select.columns

Index(['asset_id', 'title', 'text', 'start_time_local', 'media', 'entities',
       'keywords'],
      dtype='object')

In [120]:
df_select.to_parquet("20240401.parquet", engine='pyarrow')

In [101]:
df_select.head(1)

,asset_id,title,text,start_time_local,media,entities,keywords
0,105580417,Uniformados desbaratan una banda dedicada al robo de automotores,"Personal de la División Sustracción Automotores logró la detención de dos sujetos, integrantes de una banda dedicada a la sustracción de automóviles en el Gran Mendoza.\n\nEste acontecimiento es el resultado de una exhaustiva y prolongada investigación con un mismo modus operandi, la sustracción de camionetas de modelos anteriores al año 2007 debido a sus sistemas de seguridad menos sólidos.\n\nTras una pesquisa de varios meses, personal policial con el apoyo de diferentes herramientas tecnológicas, logró la identificación de una organización delictiva especializada en el robo, desarme y posterior comercialización de vehículos con documentación apócrifa en provincias vecinas.\n\nDurante la investigación se llevaron a cabo varias medidas judiciales y se logró la detención de otros integrantes de la organización, incautaron armas de fuego, dinero en efectivo y vehículos utilizados por la banda para cometer los delitos, además se encontraron herramientas relacionadas con la actividad delictiva.\n\nDinero, herramientas y artefactos de comunicación fueron secuestrados por la Policía.\n\nLas labores de vigilancia permitieron recuperar vehículos en diferentes áreas de gran Mendoza, en lugares que eran utilizados para “enfriar” los mismos para luego ser comercializarlos.\n\nLos dos detenidos, Juan C. (41) y Gastón C. (29), fueron vinculados a nueve expedientes aproximadamente en casos de hurto de vehículos en la vía pública, principalmente camionetas como Ford F-100, Ford Ranger, Chevrolet S10, Nissan e Isuzu Pick Up.\n\nLos mismos fueron aprehendidos por personal de Automotores en calle López de Gómara y Pedro del Castillo, de Guaymallén, junto a una camioneta Ford Ranger color Bordo, quienes al momento de advertir presencia de personal policial intentaron darse a la fuga en un automóvil marca Peugeot modelo 307 rojo, logrando ser detenidos y el secuestro de los bienes, trasladados a Comisaría 9na.",2024-04-05 00:00:00,Jornada Online,[Automotores],"[banda, sustracción, robo, herramientas tecnológicas, vehículos, principalmente camionetas, detención, organización delictiva, desarme, investigación, pesquisa, artefactos, expedientes, automóviles, comercialización, identificación, modelos anteriores, comisaría, labores, vigilancia, documentación, división, fuga, secuestro, bienes, sistemas, provincias, delitos, fuego, comunicación, lugares, policía, medidas judiciales, actividad, casos, apoyo, seguridad]"


In [ ]:
# Modelo 2
df_params = {'0_1000':'0_1000_data.parquet',
             '1000_2000':'1000_2000_data.parquet',
             '2000_3000':'2000_3000_data.parquet',
             'df_joined':'df_joined_2024-04-01 00_00_00.parquet'
            }

chunk = '1000_2000' 

df_parquet_2 = pd.read_parquet(PATH+df_params[chunk])
data_2 = list(df_parquet_2['in__text'])

chunk = "app_2"
# Cargar modelo entrenado o saltar celda y entrenar
topic_model_2 = BERTopic.load(PATH+f"modelos/bertopic_model_{chunk}")

# Cargar los embeddings
docs_embedding_2 = np.load(PATH+f"modelos/docs_embeddings_app.npy")

In [ ]:
# Modelo 3
chunk = "0_200_data.parquet"
df_parquet_3 = pd.read_parquet(PATH+chunk)
data_3 = list(df_parquet_3['in__text'])

chunk = "app"
# Cargar modelo entrenado o saltar celda y entrenar
topic_model = BERTopic.load(PATH+f"modelos/bertopic_model_{chunk}")

# Cargar los embeddings
#docs_embedding_1 = np.load(PATH+f"modelos/docs_embeddings_{chunk}.npy")

In [ ]:
var = topic_model_3.topics_
var_ = list(set(var))
var_

In [ ]:
len(topic_model_3.topics_)


In [ ]:
topic = -1
docs_per_topics = [i for i, x in enumerate(topic_model.topics_) if x == topic]
print(docs_per_topics)

In [ ]:
data_1[0]

In [ ]:
topics, probs = topic_model_2.transform(data_2)

#topics = np.load(PATH+f"modelos/topics_{chunk}.npy")
#probs = np.load(PATH+f"modelos/probs_{chunk}.npy")

In [ ]:
topics_to_save = list(topic_model_1.get_topics().keys())[1:]
topics_to_save


In [ ]:
topic_model_1.get_topics().keys()

In [ ]:
topic_model_2.get_topics().keys()

In [ ]:
topic_model = BERTopic.merge_models([topic_model_1, topic_model_2])
topic_model.get_topics().keys()

In [ ]:
topics, probs = topic_model.transform(data_2)

In [ ]:
len(topics)

In [ ]:
topic = 18
docs_per_topics = [i for i, x in enumerate(topic_model.topics_) if x == topic]
docs_per_topics

In [ ]:
news_total_count = 1000
docs_per_topics = [i for i, x in enumerate(topic_model.topics_) if x == topic]
docs_per_topics = [num - news_total_count for num in docs_per_topics]
docs_per_topics

In [ ]:
docs_embedding = topic_model.embedding_model.embed(data_2, verbose=True)

In [ ]:
len(docs_embedding)

In [21]:
db_news = get_news( process=True )
db_news[:1]

[['105630355',
  'Cuál es la indemnización por despido de un trabajo en España y qué cambios pide el Consejo de Europa',
  'Imagen de archivo de una oficina de empleo del SEPE. (Europa Press)\n\nEl Comité Europeo de Derechos Sociales (CEDS) declaró el pasado domingo que los despidos improcedentes en España no se ajustan a la Carta Social Europea. En dicha resolución, emitida a raíz de una reclamación colectiva del sindicato UGT en 2022, el órgano europeo con sede en Estrasburgo manifestó que la normativa española no es “suficientemente reparadora ni proporcional al daño” que producen los despidos sin causa, ya que fija de manera predeterminada y única la indemnización -33 días por año trabajado con un límite de 24 mensualidades-. Esta metodología no permite, denuncian, compensar todas las pérdidas eventuales sufridas por el trabajador.\n\nEl despido de un trabajador es la decisión unilateral -por parte de la empresa- de terminar con el contrato del empleado. Esta decisión puede ser con

In [22]:
len(db_news)

350

In [14]:
db_news_2 = get_news( '2024-04-03' )
db_news_2[:1]

[['105629872',
  'Colapso en las rutas: la mayoría regresó un día antes incentivado por chaparrones en la costa',
  'Alertados por la cantidad de turistas que eligieron localidades de la Costa Atlántica para pasar el fin de semana extra largo por Semana Santa, durante toda la jornada de hoy miles de vehículos colmaron las rutas de las region.\n\nPara mañana, último día del fin de semana extra largo, se espera, que continúe el intenso tránsito con turistas que eligen disfrutar hasta último momento las mini vacaciones.\n\nMuchos turistas decidieron regresar un día antes para evitar el caos de tránsito todavía mayor que se espera para este martes en las distintas rutas y autopistas camino a la Ciudad de Buenos Aires.\n\nLos principales servicios satelitales de tránsito marcan fuertes concentraciones en la autopista La Plata-Buenos Aires. Algunas aplicaciones recomiendan caminos alternativos.\n\n“En Ruta 11 y Autovía 2, el tránsito es intenso en sentido a CABA”, informó la concesionaria AU

In [15]:
db_plus = db_news + db_news_2


In [17]:
db_plus

[['105640870',
  'Dieron a conocer detalles del "pacto de silencio" entre la empleada y Christian Horner tras el escándalo en Red Bull que sacudió a la Fórmula 1',
  'El jefe de la escudería Red Bull de la Fórmula 1, Christian Horner, habla por teléfono antes del Gran Premio de Baréin en el circuito de Sakhir, Baréin en febrero de 2024 (REUTERS/Rula Rouhana)\n\nDesde que el caso salió a la luz, el mundo de la Fórmula 1 tiene los ojos posados sobre Red Bull Racing. Christian Horner, jefe de la escudería, fue acusado por una empleada del equipo por conducta inapropiada a través de una serie de mensajes de texto sexuales que salieron a la luz. Hasta ahora, el británico de 50 años quedó absuelto y de hecho estuvo en funciones en los primeros GP del año, pero ahora se conocieron nuevos detalles sobre el “pacto de silencio” entre las partes.\n\nLa mujer que acusó a Horner, que fue suspendida con el salario completo desde que el escándalo se conoció públicamente, fue defendida por una amiga p

In [ ]:
df_news = pd.DataFrame(db_news , columns=["indice", "titulo", "noticia", "keywords", "entidades", "creado"])

idx_data     = list(df_news.index)
id_data      = list(df_news['indice'])
title_data   = list(df_news['titulo'])
data         = list(df_news['noticia'])
keywords     = list(df_news['keywords'])
entities     = list(df_news['entidades'])

In [ ]:
len(id_data)

In [ ]:
news_total_count = 1000
topic_documents_ids, topic_documents_title, threshold  = topic_documents(topic, topic_model, probs, df_news, news_total_count)

In [ ]:
topic_documents_title

In [ ]:
threshold

In [ ]:
topic_documents_ids

In [ ]:
topic = 0
query = {
        "size": 1000,
        "query": {
            "bool": {
                "must": [
                    {   "term": {
                            "topic": topic
                        }
                    }
                ]
            }
        }
    }
                    
response = os_client.search(index='news', body=query, scroll='2m')

# Obtener el scroll ID
scroll_id = response['_scroll_id']
total_hits = response['hits']['total']['value']
all_hits = response['hits']['hits']

while len(response['hits']['hits']) > 0:
    response = os_client.scroll(scroll_id=scroll_id, scroll='2m')
    scroll_id = response['_scroll_id']
    all_hits.extend(response['hits']['hits'])

ID    = [hit['_id'] for hit in all_hits]
title = [hit['_source']['title'] for hit in all_hits]
news  = [hit['_source']['news'] for hit in all_hits]
probs = [hit['_source']['prob'] for hit in all_hits]

In [ ]:
query = {
        "size": 1000,
        "query": { "match_all": {} }
    }
                        
response = os_client.search(index='news', body=query, scroll='2m')

# Obtener el scroll ID
scroll_id = response['_scroll_id']
total_hits = response['hits']['total']['value']
all_hits = response['hits']['hits']

while len(response['hits']['hits']) > 0:
    response = os_client.scroll(scroll_id=scroll_id, scroll='2m')
    scroll_id = response['_scroll_id']
    all_hits.extend(response['hits']['hits'])

len(all_hits)

In [ ]:
def funcion_aux(ID):

    query = {
                'query': {
                    'match': {
                        '_id': ID  # Sustituir 'campo' y 'valor' por campo y valor de búsqueda
                    }
                }
    }
    
    response = os_client.search(index='news', body=query)

    # Procesar la respuesta
    results = response['hits']['hits']

    title = [ result['_source']['title'] for result in results]
    news = [ result['_source']['news'] for result in results]
    topic = [ result['_source']['topic'] for result in results]
    prob = [ result['_source']['prob'] for result in results]
    keywords = [ result['_source']['keywords'] for result in results]
    entities = [ result['_source']['entities'] for result in results]
    fila = df_parquet.index.get_loc(ID)

    try:
        keywords_df = df_parquet[df_parquet.index==ID]['Keyword Name'].values[0]
    except:
        keywords_df = ""
    try:
        entities_df = df_parquet[df_parquet.index==ID]['Entity Name'].values[0]
    except:
        entities_df = ""

    print(f"Noticia ID: {ID} {title}")
    print(f"Entities originales: {entities_df}")
    print(f"Keywords originales: {keywords_df}")
    print("-"*80)
    print(f"Topico: {topic}")
    print(f"Fila: {fila}")
    print(f"Prob. modelo: {prob}")
    print(f"Entities calculadas: {entities}")
    print(f"Keywords calculadas: {keywords}")
    print("-"*80)
    print(f"Noticia: {news[0]}")

In [ ]:
funcion_aux(105579854)

### Resetear la base de news

In [5]:
# Nombre del índice
index_name = 'news'

# Campo a actualizar y nuevo valor
campo_a_actualizar_1 = 'process'
campo_a_actualizar_2 = 'topic'
campo_a_actualizar_3 = 'prob'
nuevo_valor_1 = False
nuevo_valor_2 = -1
nuevo_valor_3 = 0

# Buscar todos los documentos en el índice
search_query = {
    "query": {
        "match_all": {}
    }
}

# Obtener todos los documentos
response = os_client.search(index=index_name, body=search_query, scroll='2m', size=1000)

documents = response['hits']['hits']
scroll_id = response['_scroll_id']

# Preparar operaciones bulk
bulk_operations = []

# Procesar el primer lote de documentos
for doc in documents:
    bulk_operations.append({
        "_op_type": "update",
        "_index": index_name,
        "_id": doc["_id"],
        "doc": {
            campo_a_actualizar_1: nuevo_valor_1,
            campo_a_actualizar_2: nuevo_valor_2,
            campo_a_actualizar_3: nuevo_valor_3
        }
    })

# Obtener y procesar el resto de los documentos con el scroll
while len(documents) > 0:
    response = os_client.scroll(scroll_id=scroll_id, scroll='2m')
    documents = response['hits']['hits']
    scroll_id = response['_scroll_id']

    for doc in documents:
        bulk_operations.append({
            "_op_type": "update",
            "_index": index_name,
            "_id": doc["_id"],
            "doc": {
                campo_a_actualizar_1: nuevo_valor_1,
                campo_a_actualizar_2: nuevo_valor_2,
                campo_a_actualizar_3: nuevo_valor_3
            }
        })

# Ejecutar las operaciones bulk
helpers.bulk(os_client, bulk_operations)

# Eliminar el scroll para liberar los recursos
os_client.clear_scroll(scroll_id=scroll_id)

{'succeeded': True, 'num_freed': 1}

In [ ]:
# Obtener todos los registros de news con sus topicos

query = {
        "size":1000,
        "query": {
            "bool": {
                "must": [
                    {"match_all": {}}
                ],
            }
        }
    }

# Ejecutar la consulta
response = os_client.search(index='news', body=query)

# Procesar la respuesta
results = response['hits']['hits']

ID = [ result['_id'] for result in results]
title = [ result['_source']['title'] for result in results]
news = [ result['_source']['news'] for result in results]
topic = [ result['_source']['topic'] for result in results]

ID = np.array(ID).reshape(-1,1)
title = np.array(title).reshape(-1,1)
news = np.array(news).reshape(-1,1)
topic = np.array(topic).reshape(-1,1)

combined_array = np.hstack((ID, title, news, topic))
df = pd.DataFrame(combined_array, columns=["id","title","text","topic"])

# Grabar df con etiquetas de topicos
file_path = 'data_clasif.xlsx'
df.to_excel(PATH+file_path, index=False)

In [ ]:
delete_index_opensearch("topic")

In [30]:
topics_name = get_topics_opensearch()
[name['name'] for name in topics_name]

['Ocupación hotelera en Córdoba.',
 'Fuga de presos en San Telmo.',
 'Violencia y conflictos internacionales',
 'Incidentes de violencia y criminalidad',
 'Temas de actualidad en Argentina.',
 'Crisis sanitaria y política en Argentina.',
 'Turismo y Deuda en Argentina',
 'Inseguridad y delitos en Córdoba.',
 'Aniversario de la Guerra de Malvinas',
 'Disminución de preocupación por inflación.']

In [102]:
from typing import Union
def dataset_validation(df: object) -> Union[bool, object]:

    columnas_necesarias = [ 'asset_id',
                            'title',
                            'text',
                            'start_time_local',
                            'media'
                            ]

    # Obtener las columnas del DataFrame
    columnas_archivo = df.columns.tolist()

    # Verificar si todas las columnas necesarias están en el archivo
    for columna in columnas_necesarias:
        if columna not in columnas_archivo:
            return False 

    if len(df) > 1000:
        # Seleccionar 1000 filas aleatorias
        df_out = df.sample(n=1000, random_state=42)
    else:
        df_out = df.copy()

    return df_out

In [33]:
if dataset_validation(df_select):
    print("Es valido")

Falta la columna: asset_id


In [34]:
from datasets import load_dataset

In [37]:
dataset = load_dataset("jganzabalseenka/news_2024-06-02_24hs")
df = pd.DataFrame(dataset['train'])

In [104]:
if isinstance(dataset_validation(df_select), bool) and not dataset_validation(df_select):
    print("Validación de columnas fallida de dataset")
else:
    print("ok")
    

ok


In [ ]:
def search_query